# Attempt with Rigging

In [ ]:
%load_ext autoreload
%autoreload 2

# Question Asker Agent

In [ ]:
# MODEL = "gpt-4o"
MODEL = "transformers!meta-llama/Meta-Llama-3-8B-Instruct,device_map=cuda:1,max_tokens=1024,load_in_4bit=True"

In [ ]:
import rigging as rg

from IPython.display import display, clear_output

import kego


class AskerQuestion(rg.Model):
    question: str

In [ ]:
questions = []
answers = []


def get_model():
    return rg.get_generator(MODEL)


async def ask_next_question(questions, answers, model, verbose=False):
    system_prompt = """
        You are playing the game 20 questions.
            - Your task is to ask questions.
            - These questions should try to quickly find the keyword.
            - You will recieve back only yes or no responses.
            - Note previous questions and their answers when creating your next question.
            - Do not assume anything specific too soon. Only as questions that divide possible answers in two evenly split groups.
            - The keyword will be a person, place or thing. Only ask questions related to these.
            - Avoid asking the question: "Is the thing you're thinking of something that can be held in your hand?".
    """
    assert len(questions) == len(answers)

    prev_content = ""
    for i, question in enumerate(questions):
        prev_content += f"question {i} <question>{question}</question> <answer>{answers[i]}</answer>"

    user_prompt = f"""
        Previous questions and answers are:
            {prev_content}
        
        Ask your question within the tag {AskerQuestion.xml_tags()}
    """
    asker = await model.chat(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    ).run()
    question = asker.last.parse(AskerQuestion).question
    if verbose:
        print(f"=== Question {len(questions) + 1} ====")
        print(question)

    return question

In [ ]:
model = get_model()

In [ ]:
kego.empty_gpu()
kego.info_gpus()

In [ ]:
# question = await ask_next_question(questions, answers, model=model)

# LLM Trying to Guess the Keyword "Tom Hanks"

In [ ]:
# answer = "no"
# questions.append(question)
# answers.append(answer)
# question = await ask_next_question(questions, answers, model=model)

In [ ]:
def print_info():
    print(f"{answers=}")
    print(f"{questions=}")


print_info()

In [ ]:
# question

In [ ]:
# answer = "no"
# questions.append(question)
# answers.append(answer)
# question = await ask_next_question(questions, answers, model=model)

# Create our Answerer Agent

In [ ]:
class YesNoAnswer(rg.Model):
    "Yes/No answer answer with coercion"
    answer: str


async def get_agent_answer(keyword, question, model):

    system_prompt = f"""
        You are playing the game 20 questions. You will be asked a question an must answer yes or no.
        Note specifically
        - Persons are living things.
        The keyword you are answering for is {keyword}
    """

    user_prompt = f"""
        The question is <question>{question}</question>
    
        Your response should be given between the tags {YesNoAnswer.xml_tags()}
    
        It should only be 'yes' or 'no'
        
        """

    answerer = await model.chat(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    ).run()

    answer = answerer.last.parse(YesNoAnswer).answer
    return answer

In [ ]:
# keyword = 'Tom Hanks'
# question = 'Is he a handsome lad?'
# await get_agent_answer(keyword, question, model=model)

# Guesser Agent

In [ ]:
class Guess(rg.Model):
    "20 Questions guess"
    guess: str


async def make_guess(
    questions: list, answers: list, guesses: list, model, verbose=False
):
    system_prompt = """
        You are playing the game 20 questions.
            - Your task is to guess the keyword.
            - Note previous questions and their answers when creating your guess
            - The keyword will be a person, place or thing.
    """
    assert len(questions) == len(answers)

    prev_content = ""
    for i, question in enumerate(questions):
        prev_content += f"question {i} <question>{question}</question> <answer>{answers[i]}</answer>"

    user_prompt = f"""
        Previous questions and answers are:
            {prev_content}

        Previous Guesses are, do not repeat these:
            {','.join(guesses)}
        
        You will now guess for the 20 question game.
          - Be specific.
          - Guess only a single thing.
        
          
        Put the word for your guess in the following tag {Guess.xml_tags()}
    """
    asker = await model.chat(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    ).run()

    guess = asker.last.parse(Guess).guess
    if verbose:
        print(f"=== Guess {len(questions) + 1} ====")
        print(guess)

    return guess

# Lets let the Agents fight it out!!!

In [ ]:
import string


def keyword_guessed(guess: str, keyword: str) -> bool:
    def normalize(s: str) -> str:
        t = str.maketrans("", "", string.punctuation)
        return s.lower().replace("the", "").replace(" ", "").translate(t)

    if normalize(guess) == normalize(keyword):
        return True
    # for s in alts:
    #   if normalize(s) == normalize(guess):
    #     return True

    return False

In [ ]:
keyword = "Tom Hanks"
questions = []
answers = []
guesses = []
max_questions = 40
for i in range(max_questions):
    print("=" * 20)
    print(f"Round {i}")
    question = await ask_next_question(questions, answers, model=model)
    print(f"[Question]: {question}")
    answer = await get_agent_answer(keyword, question, model=model)

    print(f"[Answer]:   {answer}")
    questions.append(question)
    answers.append(answer)
    guess = await make_guess(questions, answers, guesses, model=model)
    guesses.append(guess)
    print(f"[Guess]: {guess}")
    if keyword_guessed(guess, keyword):
        print("FOUND IT!!!! BIZNATCH!!!!!!")
        break